
 _____                           _  ______             _                _  _   
/  ___|                         | | | ___ \           | |              (_)| |  
\ `--.   ___   _   _  _ __    __| | | |_/ /___   _ __ | |_  _ __  __ _  _ | |_ 
 `--. \ / _ \ | | | || '_ \  / _` | |  __// _ \ | '__|| __|| '__|/ _` || || __|
/\__/ /| (_) || |_| || | | || (_| | | |  | (_) || |   | |_ | |  | (_| || || |_ 
\____/  \___/  \__,_||_| |_| \__,_| \_|   \___/ |_|    \__||_|   \__,_||_| \__|
                                                                               
                                                                               
SO, basically in this code your picture is transformed to a onesecond audio file. using a p5js sketch i turn the webcam feed into grayscale grids, assign each grid to a group, add multiple oscillators, one for each group, and the average brightness of each group makes the frequnecy data. then we train a binary model based on audios gathered from your face, against some random files. and baam the model can detect you by hearing your face!!

In [1]:
##!pip install librosa scikit-learn numpy


Feature Extraction by computing the Mel frequency cepstral coefficients 
And
Model Training
this is a simple binary classification so using a Support Vector Machine (SVM) is okay

In [3]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Feature extraction function
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfccs

# Load and label dataset
def load_dataset(directory):
    features, labels = [], []
    for filename in os.listdir(directory):
        if filename.startswith('T_'):  # 'T_' prefix for true files
            label = 1
        elif filename.startswith('F_'):  # 'F_' prefix for false files
            label = 0
        else:
            continue  
        file_path = os.path.join(directory, filename)
        features.append(extract_features(file_path))
        labels.append(label)
    return features, labels

# directory containing audio files
directory = './audio_files'
features, labels = load_dataset(directory)

# Splitting dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.25, random_state=42)

# Creating and training the model
model = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
model.fit(X_train, y_train)

# Testing the model
accuracy = model.score(X_test, y_test)
print(f'Model Accuracy: {accuracy}')


Model Accuracy: 0.8888888888888888


Prediction
predict whether a new file is similar or not based on the trained model

In [5]:
def predict_similarity(model, file_path):
    features = extract_features(file_path)
    prediction = model.predict([features])
    probability = model.predict_proba([features])[0]
    return "True" if prediction[0] == 1 else "False", probability

#Example
new_file_path = './testtrue.wav'
prediction, probability = predict_similarity(model, new_file_path)
print(f'Prediction: {prediction}, Probability: {probability}')


Prediction: True, Probability: [0.30806884 0.69193116]
